In [1]:
# import all the packages
from bs4 import BeautifulSoup
import requests

In [2]:
years = [str(x) for x in range(1965,1976)]
print(years)

['1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975']


In [3]:
wiki_url = 'https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_'

In [4]:
def hit_em_all():
    year_wise_html = {}
    for year in years:
        url = wiki_url+year
        result = requests.get(url)
        content = result.content
        year_wise_html[year] = BeautifulSoup(content, 'html.parser')
    return(year_wise_html)

In [25]:
hot_100s = hit_em_all()

In [26]:
def get_song_list_table(table):
    table_class = table.attrs['class']
    table_class_list = []
    for class_name in table_class:
        table_class_list.append(class_name.encode('ascii','ignore'))
    class_name = " ".join(table_class_list)
    return(class_name)

In [27]:
def get_the_table(page):
    tables = page.findChildren('table')
    for table in tables:
        if get_song_list_table(table) == 'wikitable sortable':
            return(table)

In [28]:
def get_all_year_song_tables():
    year_wise_table = {}
    for year, page in hot_100s.items():
        year_wise_table[year] = get_the_table(page)
    return(year_wise_table)

year_wise_table = get_all_year_song_tables()
#print(year_wise_table)

In [12]:
def get_artist_name_and_song_links():
    complete_dict = dict()
    all_artists = []
    full_song_links = []
    song_name_list = []
    wikipedia = 'https://en.wikipedia.org'
    for year, table in year_wise_table.items():
        all_tds = year_wise_table[year].find_all('td')
        artist_links = all_tds[1::2]
        for artist in artist_links:
            all_artists.append(artist.text.encode('ascii','ignore'))
        #print(year)
        #print(all_artists)
        song_links = all_tds[::2]
        for song in song_links:
            a = song.find('a')
            if a:
                link = a.attrs['href']
                print('Link',link)
                song_name_list.append(a.text.encode('ascii','ignore'))
                print('Song Name',a.text.encode('ascii','ignore'))
                full_song_links.append(wikipedia+link.encode('ascii','ignore'))
                print('Song Link',wikipedia+link.encode('ascii','ignore'))
        complete_dict[year] = [artists_links, full_song_links]
    return(complete_dict)

In [25]:
my_dict = get_artist_name_and_song_links()

('Artist', 'The Spinners')
('Link', u'/wiki/Barbra_Streisand')
('Song Name', 'Barbra Streisand')
('Song Link', 'https://en.wikipedia.org/wiki/Barbra_Streisand')
('Link', u'/wiki/Seasons_in_the_Sun')
('Song Name', 'Seasons in the Sun')
('Song Link', 'https://en.wikipedia.org/wiki/Seasons_in_the_Sun')
('Link', u'/wiki/Love_Unlimited_Orchestra')
('Song Name', 'Love Unlimited Orchestra')
('Song Link', 'https://en.wikipedia.org/wiki/Love_Unlimited_Orchestra')
('Link', u'/wiki/Come_and_Get_Your_Love')
('Song Name', 'Come and Get Your Love')
('Song Link', 'https://en.wikipedia.org/wiki/Come_and_Get_Your_Love')
('Link', u'/wiki/The_Jackson_5')
('Song Name', 'The Jackson 5')
('Song Link', 'https://en.wikipedia.org/wiki/The_Jackson_5')
('Link', u'/wiki/The_Loco-Motion')
('Song Name', 'The Loco-Motion')
('Song Link', 'https://en.wikipedia.org/wiki/The_Loco-Motion')
('Link', u'/wiki/MFSB')
('Song Name', 'MFSB')
('Song Link', 'https://en.wikipedia.org/wiki/MFSB')
('Link', u'/wiki/The_Streak')
('Son

NameError: global name 'artists_links' is not defined

In [ ]:
def get_full_artist_links():
    all_artists = []
    for artist in artist_links:
        all_artists.append(artist.text.encode('ascii','ignore'))
    return(all_artists)

def get_full_song_links():
    full_song_links = []
    song_name_list = []
    wikipedia = 'https://en.wikipedia.org'
    for song in song_links:
        a = song.find('a')
        link = a.attrs['href']
        song_name_list.append(a.text.encode('ascii','ignore'))
        full_song_links.append(wikipedia+link.encode('ascii','ignore'))
    return(full_song_links,song_name_list)

In [60]:
#### End of experimental automation

In [78]:
all_tds = year_wise_table['1965'].find_all('td')

In [85]:
def data_pipeline(year):
    year = ''+str(year)
    all_tds = year_wise_table[year].find_all('td')

    my_tds = []
    for td in all_tds:
        if not td.text.isdigit():
            my_tds.append(td)

    artist_links = my_tds[1::2]
    song_links = my_tds[::2]

    def get_full_artist_links():
        all_artists = []
        for artist in artist_links:
            if not artist.text.isdigit():
                all_artists.append(artist.text.encode('ascii','ignore'))
        return(all_artists)

    artists = get_full_artist_links()

    def get_full_song_links():
        full_song_links = []
        song_name_list = []
        wikipedia = 'https://en.wikipedia.org'
        for song in song_links:
            #print(song)
            a = song.find('a')
            #print(a)
            if a:
                link = a.attrs['href']
                song_name_list.append(a.text.encode('ascii','ignore'))
                full_song_links.append(wikipedia+link.encode('ascii','ignore'))
        return(full_song_links,song_name_list)

    links, songs = get_full_song_links()

    table = pd.DataFrame(list(zip([1990]*min(len(artists),len(songs),len(links)),songs, artists, links)), columns=['Year','Song','Artist','Song Link'])

    table.head(5)

    def get_the_right_table(table):
        attributes = table.attrs
        table_class_list = []
        class_name=''
        if 'class' in attributes:
            table_classes = table.attrs['class']
            for class_name in table_classes:
                table_class_list.append(class_name.encode('ascii','ignore'))
            class_name = " ".join(table_class_list)
            return(class_name)

    def get_genres(table):
        genre_dict = dict()
        songs = []
        for index, row in table.iterrows():
            #print('Song',row['Song'])
            #print('Artist',row['Artist'])
            try:
                result = requests.get(row['Song Link'])
                content = result.content
                html = BeautifulSoup(content, 'html.parser')
                tables = html.findChildren('table')
                my_table = ''
                for table in tables:
                    if get_the_right_table(table) == "infobox vevent":
                        my_table = table
                        all_as = [a.text.encode('ascii','ignore') for a in my_table.find_all('a')]
                        genres = all_as[all_as.index('Genre')+1:all_as.index('Label')]
                        # the solution is from https://stackoverflow.com/questions/20585920/how-to-add-multiple-values-to-a-dictionary-key-in-python
                        key = row['Song']
                        #print(row['Song'])
                        genre_dict.setdefault(key, [])
                        genre_dict[key].append(genres)
                        #print(genre_dict[key])
            except Exception:
                pass
        return(genre_dict)

    #if hasNumbers(a.text.encode('ascii','ignore'))
    genre_dict = get_genres(table)

    len(genre_dict)

    genre_list = []
    # flatten out the genres list which is multidimensional
    for genre in genre_dict.values():
        genres = [x for sublist in genre for x in sublist]
        genre_list.append(genres)

    df_genres = pd.DataFrame(np.column_stack([genre_dict.keys(), genre_list]), 
                                   columns=['Song','Genres'])

    df_genres.head(5)

    import re
    def remove_numbers(my_list):
        final_genres = []
        for x in my_list:
            if not re.search('\d+', x):
                final_genres.append(x.title())
        return(",".join(final_genres))

    df = df_genres

    df['Final Genres'] = df['Genres'].apply(remove_numbers)

    del df['Genres']

    df.head(5)
    return(df)

In [86]:
year = range(1965,2016)

In [87]:
for y in year:
    df = data_pipeline(y)
    a = str(y)+'.csv'
    df.to_csv(a)

In [13]:
all_tds = year_wise_table['1967'].find_all('td')

In [15]:
def get_my_tds():
    my_tds = []
    for td in all_tds:
        if not td.text.isdigit():
            my_tds.append(td)

In [17]:
artist_links = my_tds[1::2]
song_links = my_tds[::2]

In [21]:
def get_full_artist_links():
    all_artists = []
    for artist in artist_links:
        if not artist.text.isdigit():
            all_artists.append(artist.text.encode('ascii','ignore'))
    return(all_artists)

In [22]:
artists = get_full_artist_links()

In [24]:
def get_full_song_links():
    full_song_links = []
    song_name_list = []
    wikipedia = 'https://en.wikipedia.org'
    for song in song_links:
        #print(song)
        a = song.find('a')
        #print(a)
        if a:
            link = a.attrs['href']
            song_name_list.append(a.text.encode('ascii','ignore'))
            full_song_links.append(wikipedia+link.encode('ascii','ignore'))
    return(full_song_links,song_name_list)

In [25]:
links, songs = get_full_song_links()

<td>"<a href="/wiki/To_Sir_With_Love_(song)" title="To Sir With Love (song)">To Sir With Love</a>"</td>
<a href="/wiki/To_Sir_With_Love_(song)" title="To Sir With Love (song)">To Sir With Love</a>
<td>"<a href="/wiki/The_Letter_(The_Box_Tops_song)" title="The Letter (The Box Tops song)">The Letter</a>"</td>
<a href="/wiki/The_Letter_(The_Box_Tops_song)" title="The Letter (The Box Tops song)">The Letter</a>
<td>"<a href="/wiki/Ode_to_Billie_Joe" title="Ode to Billie Joe">Ode to Billie Joe</a>"</td>
<a href="/wiki/Ode_to_Billie_Joe" title="Ode to Billie Joe">Ode to Billie Joe</a>
<td>"<a href="/wiki/Windy" title="Windy">Windy</a>"</td>
<a href="/wiki/Windy" title="Windy">Windy</a>
<td>"<a href="/wiki/I%27m_a_Believer" title="I'm a Believer">I'm a Believer</a>"</td>
<a href="/wiki/I%27m_a_Believer" title="I'm a Believer">I'm a Believer</a>
<td>"<a href="/wiki/Light_My_Fire" title="Light My Fire">Light My Fire</a>"</td>
<a href="/wiki/Light_My_Fire" title="Light My Fire">Light My Fire</a>


In [26]:
import pandas as pd
import numpy as np

In [32]:
table = pd.DataFrame(list(zip([1990]*min(len(artists),len(songs),len(links)),songs, artists, links)), columns=['Year','Song','Artist','Song Link'])

In [34]:
def get_the_right_table(table):
    attributes = table.attrs
    table_class_list = []
    class_name=''
    if 'class' in attributes:
        table_classes = table.attrs['class']
        for class_name in table_classes:
            table_class_list.append(class_name.encode('ascii','ignore'))
        class_name = " ".join(table_class_list)
        return(class_name)

In [55]:
#for index, row in table.iterrows():
#    print('Artist',row['Artist'])
#    print('Song',row['Song'])

In [37]:
def get_genres(table):
    genre_dict = dict()
    songs = []
    for index, row in table.iterrows():
        #print('Song',row['Song'])
        #print('Artist',row['Artist'])
        try:
            result = requests.get(row['Song Link'])
            content = result.content
            html = BeautifulSoup(content, 'html.parser')
            tables = html.findChildren('table')
            my_table = ''
            for table in tables:
                if get_the_right_table(table) == "infobox vevent":
                    my_table = table
                    all_as = [a.text.encode('ascii','ignore') for a in my_table.find_all('a')]
                    genres = all_as[all_as.index('Genre')+1:all_as.index('Label')]
                    # the solution is from https://stackoverflow.com/questions/20585920/how-to-add-multiple-values-to-a-dictionary-key-in-python
                    key = row['Song']
                    #print(row['Song'])
                    genre_dict.setdefault(key, [])
                    genre_dict[key].append(genres)
                    #print(genre_dict[key])
        except Exception:
            pass
    return(genre_dict)

In [38]:
#if hasNumbers(a.text.encode('ascii','ignore'))
genre_dict = get_genres(table)

In [39]:
len(genre_dict)

87

In [40]:
genre_list = []
# flatten out the genres list which is multidimensional
for genre in genre_dict.values():
    genres = [x for sublist in genre for x in sublist]
    genre_list.append(genres)

In [41]:
df_genres = pd.DataFrame(np.column_stack([genre_dict.keys(), genre_list]), 
                               columns=['Song','Genres'])

In [56]:
df_genres.head(5)

,Song,Final Genres
0,Everlasting Love,"Soul,Pop,Disco,Pop,Pop,Electropop,Dance"
1,Baby I Love You,Soul
2,Dedicated to the One I Love,"Pop,Soul,Pop"
3,For What It's Worth,"Folk Rock,Country Rock,Psychedelic Rock"
4,Here Comes My Baby,Beat


In [43]:
# df_genres = pd.DataFrame(list(genre_dict.items()), columns=['Song', 'Genres'])

In [44]:
df_genres.dtypes

Song      object
Genres    object
dtype: object

In [45]:
df_genres.loc[2:5,'Genres']

2                                     [Pop, soul, Pop]
3    [Folk rock, [1], country rock, [2], psychedeli...
4                                               [Beat]
5                              [Psychedelic rock, [1]]
Name: Genres, dtype: object

In [48]:
import re
def remove_numbers(my_list):
    final_genres = []
    for x in my_list:
        if not re.search('\d+', x):
            final_genres.append(x.title())
    return(",".join(final_genres))

In [49]:
df = df_genres

In [50]:
df['Final Genres'] = df['Genres'].apply(remove_numbers)

In [51]:
df.head(4)

,Song,Genres,Final Genres
0,Everlasting Love,"[Soul, Pop, Disco, Pop, Pop, Electropop, dance]","Soul,Pop,Disco,Pop,Pop,Electropop,Dance"
1,Baby I Love You,[Soul],Soul
2,Dedicated to the One I Love,"[Pop, soul, Pop]","Pop,Soul,Pop"
3,For What It's Worth,"[Folk rock, [1], country rock, [2], psychedeli...","Folk Rock,Country Rock,Psychedelic Rock"


In [52]:
del df['Genres']

In [53]:
df.head(5)

,Song,Final Genres
0,Everlasting Love,"Soul,Pop,Disco,Pop,Pop,Electropop,Dance"
1,Baby I Love You,Soul
2,Dedicated to the One I Love,"Pop,Soul,Pop"
3,For What It's Worth,"Folk Rock,Country Rock,Psychedelic Rock"
4,Here Comes My Baby,Beat


In [54]:
df.to_csv('song_genres_1968.csv')

In [151]:
# [type(x) for x in df['Genres']]

In [ ]:
# reference
# https://codereview.stackexchange.com/questions/55756/tough-list-comprehension-split-letters-and-numbers
# https://spapas.github.io/2016/04/27/python-nested-list-comprehensions/
# http://jonathansoma.com/lede/foundations/classes/pandas%20columns%20and%20functions/apply-a-function-to-every-row-in-a-pandas-dataframe/

In [240]:
# [x for x in df['Genres'] if not any(str.isdigit(c) for c in x)]
#[", ".join(x) for x in df['Genres'] if not any(str.isdigit(c) for c in x)]

In [50]:
### The next line is some awesome shit

In [ ]:
songs = genre_dict.keys()
song_genres = genre_dict.values()
genre_list = []
# flatten out the genres list which is multidimensional
for genre,song in zip(song_genres,songs):
    genres = [x for sublist in genre for x in sublist]
    for genre in genres:
        while not any(str.isdigit(c) for c in genre):
            #print(genre)
            genre_list.append(genre)
    genre_list.append(genres)

In [227]:
result = requests.get('https://en.wikipedia.org/wiki/Let_It_Go_(Disney_song)')
content = result.content
html = BeautifulSoup(content, 'html.parser')

In [228]:
tables = html.findChildren('table')

In [229]:
len(tables)

27

In [230]:
def get_the_right_table(table):
    attributes = table.attrs
    table_class_list = []
    class_name=''
    if 'class' in attributes:
        table_classes = table.attrs['class']
        for class_name in table_classes:
            table_class_list.append(class_name.encode('ascii','ignore'))
        class_name = " ".join(table_class_list)
        return(class_name)

In [ ]:
sicko = []
for table in tables:
    if get_the_right_table(table) == "infobox vevent":
        print('yes')
        sicko.

In [244]:
sicko = ''
for table in tables:
    if get_the_right_table(table) == "infobox vevent":
        all_as = [a.text.encode('ascii','ignore') for a in table.find_all('a')]
        print('hello hello hello')
        print(all_as)

hello hello hello
['Song', 'Idina Menzel', 'Frozen', 'Wonderland Music Company', '[1]', 'Label', 'Walt Disney', 'Songwriter(s)', 'Kristen Anderson-Lopez', 'Robert Lopez', 'Frozen', 'Love Is an Open Door', 'Reindeer(s) Are Better Than People', '"Let It Go"', 'YouTube']
hello hello hello
['', 'Single', 'Demi Lovato', 'Frozen', 'Digital download', '[1]', 'Genre', 'Pop rock', 'Label', 'Walt Disney', 'Songwriter(s)', 'Kristen Anderson-Lopez', 'Robert Lopez', 'Producer(s)', 'Emanuel Kiriakou', 'Andrew Goldstein', 'Demi Lovato', 'Made in the USA', 'Neon Lights', 'Made in the USA', 'Neon Lights', '"Let It Go"', 'YouTube']
hello hello hello
['', 'Single', 'Idina Menzel', 'Frozen', 'Digital download', '[1]', 'Genre', 'Show tune', 'Label', 'Walt Disney', 'Songwriter(s)', 'Kristen Anderson-Lopez', 'Robert Lopez', 'Producer(s)', 'Kristen Anderson-Lopez', 'Robert Lopez', 'Christophe Beck', 'Idina Menzel', 'Next to Me', 'Next to Me']


In [236]:
sicko.attrs

{u'class': [u'infobox', u'vevent'], u'style': u'width:22em'}

In [241]:
all_as = [a.text.encode('ascii','ignore') for a in sicko.find_all('a')]

In [238]:
genres = all_as[all_as.index('Genre')+1:all_as.index('Label')]

In [130]:
table = html.findAll("table", {"class": "infobox vevent"})

In [137]:
all_trs = table.source.searchTag('a)

In [138]:
all_t

In [ ]:
genres = {}
for html in html_list:
    info_div = happy.findAll("div", {"class": "hlist hlist-separated"})
    our_div = info_div[1]
    li = our_div.findAll('li')
    for item in li:
        print(item.text)

In [ ]:
## First code

In [487]:
result = requests.get(hot_100)
c = result.content
soup = BeautifulSoup(c, 'html.parser')

In [415]:
tables = soup.findChildren('table')

In [383]:
my_table = tables[0]

In [403]:
table_class = my_table.attrs['class']

In [412]:
table_class_list = []
for class_name in table_class:
    table_class_list.append(class_name.encode('ascii','ignore'))    

In [414]:
" ".join(table_class_list)

'wikitable sortable'

In [503]:
type(a_t)

bs4.element.Tag

In [543]:
a = a_t.find('a')

In [544]:
type(a)

bs4.element.Tag

In [545]:
att = a.attrs['href']

In [291]:
a_content = []
for td in all_tds:
    a_content.append(td.)

In [293]:
a_t = a_content[0]

In [294]:
type(a_t)

list

In [295]:
a_t

[u'"',
 <a href="/wiki/Happy_(Pharrell_Williams_song)" title="Happy (Pharrell Williams song)">Happy</a>,
 u'"']

In [280]:
len(a)

200

In [281]:
a[0]

<bound method Tag.find_all of <td>"<a href="/wiki/Happy_(Pharrell_Williams_song)" title="Happy (Pharrell Williams song)">Happy</a>"</td>>

In [282]:
a0 = a[0]

In [277]:
type(a)

list

In [231]:
rows = my_table.findChildren('td')

In [232]:
len(rows)

200

In [249]:
anchors_100 = []
for row in rows:
    anchors_100.append(row.findChildren('a'))

In [252]:
a = anchors_100[0]

In [259]:
a.source.search_tag('href')

In [166]:
song_links = []
for link in links:
    href = link.attrs['href']
    print(href)
    if ['_(song)','_(singer)','_(band)' or in href:
        song_links.append('https://en.wikipedia.org'+href)

/wiki/Billboard_(magazine)
/wiki/Billboard_Year-End
/wiki/Billboard_Hot_100
#cite_note-2
/wiki/Happy_(Pharrell_Williams_song)
/wiki/Pharrell_Williams
/wiki/Dark_Horse_(Katy_Perry_song)
/wiki/Katy_Perry
/wiki/Juicy_J
/wiki/All_of_Me_(John_Legend_song)
/wiki/John_Legend
/wiki/Fancy_(Iggy_Azalea_song)
/wiki/Iggy_Azalea
/wiki/Charli_XCX
/wiki/Counting_Stars
/wiki/OneRepublic
/wiki/Talk_Dirty_(Jason_Derulo_song)
/wiki/Jason_Derulo
/wiki/2_Chainz
/wiki/Rude_(song)
/wiki/Magic!
/wiki/All_About_That_Bass
/wiki/Meghan_Trainor
/wiki/Problem_(Ariana_Grande_song)
/wiki/Ariana_Grande
/wiki/Iggy_Azalea
/wiki/Stay_with_Me_(Sam_Smith_song)
/wiki/Sam_Smith_(singer)
/wiki/Timber_(Pitbull_song)
/wiki/Pitbull_(rapper)
/wiki/Kesha
/wiki/Pompeii_(Bastille_song)
/wiki/Bastille_(band)
/wiki/Shake_It_Off_(Taylor_Swift_song)
/wiki/Taylor_Swift
/wiki/Am_I_Wrong_(song)
/wiki/Nico_%26_Vinz
/wiki/Turn_Down_for_What
/wiki/DJ_Snake
/wiki/Lil_Jon
/wiki/The_Monster_(Eminem_song)
/wiki/Eminem
/wiki/Rihanna
/wiki/Say_Som

In [14]:
type(song_links)

list

In [23]:
html_list = []
for song in song_links:
    result = requests.get(song)
    c = result.content
    html = BeautifulSoup(c, 'html.parser')
    html_list.append(html)

In [155]:
len(html_list)

50

In [156]:
genres = {}
for html in html_list:
    info_div = happy.findAll("div", {"class": "hlist hlist-separated"})
    our_div = info_div[1]
    li = our_div.findAll('li')
    for item in li:
        print(item.text)

Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul
Soul
neo soul


In [120]:
url = 'https://en.wikipedia.org/wiki/All_About_That_Bass'

In [121]:
result = requests.get(url)
c = result.content
html = BeautifulSoup(c, 'html.parser')

In [123]:
info_div = html.findAll("div", {"class": "hlist hlist-separated"})
print(len(info_div))
our_div = info_div[1]
li = our_div.findAll('li')
for item in li:
    print(item.text)

0


IndexError: list index out of range

In [167]:
happy = html_list[0]

In [168]:
info_div = happy.findAll("div", {"class": "hlist hlist-separated"})

In [169]:
len(info_div)

3

In [170]:
our_div = info_div[1]

In [171]:
li = our_div.findAll('li')

In [172]:
chumma =','
for item in li:
    print(item.text)
    chumma.join(item.text)

Soul
neo soul


In [45]:
mydivs = soup.findAll('div')
for div in mydivs:
    print(div['class'])

[u'noprint']
[u'noprint']
[u'mw-body']
[u'mw-body-content']
[u'mw-indicators', u'mw-body-content']
[u'mw-body-content']
[u'noprint']


KeyError: 'class'

In [51]:
obj = soup.find_all("th", text="GENRE :").find_next_sibling("td").text

AttributeError: ResultSet object has no attribute 'find_next_sibling'. You're probably treating a list of items like a single item. Did you call find_all() when you meant to call find()?

In [74]:
table = soup.findAll("table", {"class": "infobox vevent"})

In [73]:
div.source.search

<bound method SoupStrainer.search of <bs4.element.SoupStrainer object at 0x000000001DD088D0>>